In [2]:
import cv2
import numpy as np
import scipy.spatial.distance as distance
from sklearn.preprocessing import normalize
import pandas as pd
import time
from sklearn.cluster import KMeans

In [ ]:
database = []
for image in range(1, 257):
    descriptor = np.loadtxt("data/sifts/sift1to256/im" + str(image) + ".txt")
    database.append(descriptor)
print(database[0])

In [3]:
f = open("data/centroids/K8.txt", "r")
matrix = []
for line in f:
    vector = [float(x) for x in line.split()]
    matrix.append(vector)
k8_matrix = np.asmatrix(matrix)
k8 = pd.DataFrame(k8_matrix)
k8

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,31.249016,25.501215,22.848219,18.480101,13.627398,6.908546,5.087223,10.563511,42.866863,55.474236,...,56.038082,36.700588,29.084457,8.825639,5.238123,10.136950,21.843779,20.385212,18.016674,23.469543
1,21.290874,10.059727,6.766247,6.227464,11.707767,23.591778,28.157183,20.995457,18.652391,9.034462,...,6.547870,10.266053,32.679180,22.506399,13.920855,10.740374,10.970414,8.062580,7.733337,14.385700
2,30.964367,22.360004,11.048914,7.539116,8.339045,7.150357,7.396674,12.932173,29.042273,58.728218,...,78.653633,29.315863,13.792577,8.202235,6.326416,5.552842,10.343047,25.276258,30.171139,17.270981
3,78.423637,15.448256,2.299043,2.417652,7.451628,8.142867,9.004931,23.832005,94.285484,34.447968,...,88.080315,24.403255,21.304554,12.244978,7.751577,2.764812,4.603053,22.725615,36.583580,18.503300
4,6.162910,3.690419,5.116338,9.490084,14.890289,10.904242,14.217042,9.522003,26.232443,6.069299,...,11.523263,8.624854,24.377153,21.861450,17.800270,9.168837,7.446078,5.518549,7.415652,10.327967
5,26.204842,12.373660,6.769173,2.924230,5.621014,22.741890,36.099194,21.301987,21.546745,12.722086,...,6.244895,23.427507,74.618210,15.956566,2.682510,3.018473,7.716383,8.938708,9.732193,24.006351
6,24.486942,24.795143,21.648453,12.421943,7.649964,4.603993,6.157518,9.380072,21.086330,50.338490,...,18.277050,26.626511,6.380827,3.303993,4.450360,12.207950,18.904928,11.086007,10.085719,8.916367
7,16.922964,5.360461,5.361770,17.440796,29.278240,16.435246,13.679707,17.318722,24.396648,4.980571,...,4.613878,12.666457,32.459858,27.173973,31.248337,26.757372,13.805552,5.197225,4.170044,11.380675


In [4]:
f = open("data/centroids/K64.txt", "r")
matrix = []
for line in f:
    vector = [float(x) for x in line.split()]
    matrix.append(vector)
k64_matrix = np.asmatrix(matrix)
k64 = pd.DataFrame(k64_matrix)
k64

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,55.017387,27.732914,25.589029,22.385492,5.846823,1.271883,1.856415,16.184053,52.176258,43.054256,...,15.794964,26.583633,5.542566,1.874400,3.212830,11.479317,13.895384,8.611211,9.198442,11.030576
1,9.750383,8.076648,7.122637,6.057230,10.505110,26.380939,23.387072,10.596577,9.492335,7.537813,...,5.995401,8.767246,33.122635,22.427185,8.837762,5.223812,5.956311,5.985437,7.366122,13.871231
2,11.615864,5.338760,7.013163,26.854174,33.720470,13.098025,8.919293,12.362660,31.318670,5.047454,...,3.358504,6.695532,19.701420,35.112572,39.463455,31.199514,15.660893,5.565293,3.372359,5.937652
3,27.294710,20.639217,9.167409,6.584189,8.757411,6.819221,6.512691,9.588840,30.249563,73.050377,...,98.767876,54.701801,12.201124,3.907964,2.948072,2.338694,4.863980,22.243557,60.295486,28.328812
4,17.189920,31.199106,37.642750,28.723684,14.170308,4.597567,2.817527,4.340119,25.837885,80.101044,...,30.836891,36.236595,25.819761,7.784012,5.207547,12.295680,27.527309,16.763159,11.465243,19.496277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,90.458786,18.469360,2.667299,1.991594,3.959056,3.473970,5.240239,21.454447,92.725868,33.982918,...,76.442245,21.890726,13.894794,10.558840,11.190619,6.071584,7.682484,16.777657,22.223427,12.309381
60,6.490557,2.958544,4.345002,11.022571,9.424689,6.867803,8.607094,11.404882,25.599262,6.943805,...,2.931368,10.478121,51.889450,26.575312,21.081530,11.269921,4.602027,2.516813,3.255182,18.911100
61,28.055754,13.739642,3.393862,1.748338,5.594885,33.572380,37.724808,23.291048,15.449616,10.139642,...,18.110485,31.877237,34.552940,10.662915,2.693606,2.093095,5.267519,22.178516,34.865475,33.396931
62,41.782928,9.518311,3.621087,8.445363,32.001183,19.354696,9.300650,14.437980,104.987007,31.477259,...,119.804787,31.513586,11.873893,4.407265,2.435027,1.002953,3.885706,43.675724,78.267868,27.064974


In [5]:
def BoW(requests, database, clusters):
    K = clusters.shape[0]
    kmeans_model = KMeans(n_clusters=K)
    kmeans_model.cluster_centers_ = clusters
    results = []
    for request in requests:
        results.append(kmeans_model.predict(database[request].reshape(1, -1)))
results = BoW(requests, nbow, k8)

NameError: name 'requests' is not defined